## Практика ML-7. Оптимизация гиперпараметров модели

### Задача. Необходимо предсказать биологический ответ молекул (столбец 'Activity') по их химическому составу (столбцы D1-D1776).

In [1]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
from sklearn import linear_model #линейные модели
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
#from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
import optuna

import warnings
warnings.filterwarnings('ignore')

random_state = 42

data = pd.read_csv('../data/_train_sem09 (1).csv')
data.head()

c:\python\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


Так как  данные уже закодированы и нормализованы, переходим к созданию матрицы наблюдений $X$ и вектора ответов $y$

In [2]:
X = data.drop(['Activity'], axis=1)
y = data['Activity']

Разделяем выборку на тренировочную и тестовую в соотношении 80/20.

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.2)

### <center>  **Логистическая регрессия**

Сначала посчитаем модель с параметрами по умолчанию и оценим метрику f1

In [4]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(max_iter = 1000, random_state=random_state)
#Обучаем модель
log_reg.fit(X_train, y_train)
#print("accuracy на тестовом наборе: {:.2f}".format(log_reg.score(X_test, y_test)))
y_test_pred = log_reg.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.78


### **GridSearchCV**

In [5]:
param_grid = {'penalty': ['l2', 'none'] ,  #тип регурялизации
              'solver': ['lbfgs', 'saga'], #алгоритм оптимизации
            }

grid_search = GridSearchCV(
        estimator=linear_model.LogisticRegression(
        random_state=random_state, #генератор случайных чисел
        max_iter=1000 #количество итераций на сходимость
    ), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search.fit(X_train, y_train) 
#print("accuracy на тестовом наборе: {:.2f}".format(grid_search.score(X_test, y_test)))
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

CPU times: total: 9.27 s
Wall time: 3min 16s
f1_score на тестовом наборе: 0.78
Наилучшие значения гиперпараметров: {'penalty': 'l2', 'solver': 'lbfgs'}


Значение метрики f1 не изменилось, т.е. мы не нашли комбинацию внешних параметров лучше, чем заданы по умолчанию.

### **RandomizedSearchCV**

In [6]:
param_distributions = {'penalty': ['l2', 'none'] ,
                    'solver': ['lbfgs', 'sag'],
                    'C': list(np.linspace(0.01, 1, 10, dtype=float)) #np.linspace(start(от), stop(до), num=50(количество),dtype- тип данных)
                } 
            
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=random_state, max_iter=1000), 
    param_distributions=param_distributions, 
    cv=5,  
    n_iter=10,   
    n_jobs = -1
) 
 
%time random_search.fit(X_train, y_train) 
#print("accuracy на тестовом наборе: {:.2f}".format(random_search.score(X_test, y_test)))
y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

CPU times: total: 4.28 s
Wall time: 3min 53s
f1_score на тестовом наборе: 0.80
Наилучшие значения гиперпараметров: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.12}


С помощью метода базовой оптимизации RandomizedSearchCV удалось немного улучшить метрику f1.

### Hyperopt (продвинутая оптимизация)

In [7]:
space = {'penalty': hp.choice('penalty', ["none", "l2"]),
        'solver': hp.choice('solver', ["lbfgs", "sag"]),
        'C': hp.uniform('C', 0.01, 1)
        }

def hyperopt_lr(params, cv=5, X=X_train, y=y_train, random_state=random_state):    
    model = linear_model.LogisticRegression(**params, random_state=random_state, max_iter=50)
    # обучаем модель
    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))
    return -score

In [8]:
%%time

trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_lr, # наша функция 
        space=space, # пространство гиперпараметров
        algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
        max_evals=50, # максимальное количество итераций
        trials=trials, # логирование результатов
        rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
        )

print("Наилучшие значения гиперпараметров {}".format(best))

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 50/50 [00:52<00:00,  1.05s/trial, best loss: -0.8887537993920973]
Наилучшие значения гиперпараметров {'C': 0.08292441519601887, 'penalty': 0, 'solver': 0}
CPU times: total: 1min 14s
Wall time: 52.4 s


In [9]:
# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(
    random_state=42, 
    #max_iter=50,
    C=float(best['C'])
    #penalty=int(best['penalty']),
    #solver=int(best['solver'])
)
model.fit(X_train, y_train)
#print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.80


Значение метрики f1 такое же как при использовании метода RandomizedSearchCV, но было затрачено меньше времени.

### Optuna (продвинутая оптимизация)

In [10]:
def optuna_lr(trial):
      # задаем пространства поиска гиперпараметров
      penalty_ = trial.suggest_categorical('penalty', ['l2', 'none'])
      solver_ = trial.suggest_categorical('solver', ["lbfgs", "sag"])
      C_ = trial.suggest_float("C", 0.01, 1, step=0.01)

      # создаем модель
      model = linear_model.LogisticRegression(penalty=penalty_,
                                          solver=solver_,
                                          C=C_,
                                          random_state=42,
                                          max_iter=50)
      # обучаем модель
      model.fit(X_train, y_train)
      score = metrics.f1_score(y_train, model.predict(X_train))
      return score

In [11]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_lr, n_trials=20)

[I 2023-07-18 16:30:17,289] A new study created in memory with name: LogisticRegression
[I 2023-07-18 16:30:19,397] Trial 0 finished with value: 0.8732137427789601 and parameters: {'penalty': 'l2', 'solver': 'sag', 'C': 0.53}. Best is trial 0 with value: 0.8732137427789601.
[I 2023-07-18 16:30:21,498] Trial 1 finished with value: 0.8782265411478896 and parameters: {'penalty': 'none', 'solver': 'sag', 'C': 0.46}. Best is trial 1 with value: 0.8782265411478896.
[I 2023-07-18 16:30:21,939] Trial 2 finished with value: 0.8887537993920973 and parameters: {'penalty': 'none', 'solver': 'lbfgs', 'C': 0.5}. Best is trial 2 with value: 0.8887537993920973.
[I 2023-07-18 16:30:22,375] Trial 3 finished with value: 0.8887537993920973 and parameters: {'penalty': 'none', 'solver': 'lbfgs', 'C': 0.06999999999999999}. Best is trial 2 with value: 0.8887537993920973.
[I 2023-07-18 16:30:24,446] Trial 4 finished with value: 0.8748481166464156 and parameters: {'penalty': 'l2', 'solver': 'sag', 'C': 0.63}. B

CPU times: total: 26.8 s
Wall time: 17.2 s


In [12]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
#print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))
# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(**study.best_params, random_state=random_state)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Наилучшие значения гиперпараметров {'penalty': 'none', 'solver': 'lbfgs', 'C': 0.5}
f1_score на тестовом наборе: 0.75


Не удалось улучшить метрику f1 с помощью метода Optuna.

**Вывод по методам оптимизации для модели логистической регрессии.**

Метрику f1 для логистической регрессии удалось улучшить с помощью методов RandomizedSearchCV и Hyperopt

### <center> **Случайный лес**

Сначала посчитаем модель с параметрами по умолчанию и оценим метрику f1

In [13]:
#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=42)
#Обучаем модель
rf.fit(X_train, y_train)
#Выводим значения метрики 
y_test_pred = rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.82


### **GridSearchCV**

In [14]:
param_grid = {'n_estimators': list(range(80, 200, 30)),
              'min_samples_leaf': [5],
              'max_depth': list(np.linspace(20, 40, 10, dtype=int))
            }
            
grid_search_forest = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search_forest.fit(X_train, y_train) 
#print("accuracy на тестовом наборе: {:.2f}".format(grid_search_forest.score(X_test, y_test)))
y_test_pred = grid_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_forest.best_params_))

CPU times: total: 4.55 s
Wall time: 2min 52s
f1_score на тестовом наборе: 0.81
Наилучшие значения гиперпараметров: {'max_depth': 26, 'min_samples_leaf': 5, 'n_estimators': 170}


Значение метрики f1 уменьшилось, т.е. мы не нашли комбинацию внешних параметров лучше, чем заданы по умолчанию.

### **RandomizedSearchCV**

In [15]:
# Задаем сетку гиперпараметров:
param_distributions = {'n_estimators': list(range(80, 200, 30)), # количество деревьев в лесу, по умолчанию =100
              'min_samples_leaf': [5],                              # минимальное количество объектов в листе
              'max_depth': list(np.linspace(20, 40, 10, dtype=int)) # максимальная глубина дерева
              }
            
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions, 
    cv=5,
    n_iter = 10, 
    n_jobs = -1
)  
%time random_search_forest.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(random_search_forest.score(X_test, y_test)))
y_test_pred = random_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))

CPU times: total: 3.81 s
Wall time: 46.6 s
accuracy на тестовом наборе: 0.79
f1_score на тестовом наборе: 0.81
Наилучшие значения гиперпараметров: {'n_estimators': 170, 'min_samples_leaf': 5, 'max_depth': 26}


Метрику f1 не удалось улучшить с помощью метода RandomizedSearchCV.

### Hyperopt (продвинутая оптимизация)

In [16]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 300, 10),
       'max_depth' : hp.quniform('max_depth', 15, 40, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 3, 7, 1)
      }

In [17]:
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=42):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=42)
    # обучаем модель
    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))    
    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [18]:
%%time
random_state=42
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=10, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate = np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 10/10 [00:43<00:00,  4.38s/trial, best loss: -0.974969474969475]
Наилучшие значения гиперпараметров {'max_depth': 22.0, 'min_samples_leaf': 3.0, 'n_estimators': 200.0}
CPU times: total: 43 s
Wall time: 43.8 s


In [19]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=42, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train, y_train)
#print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.82


Получили значение метрики f1 такое же, как при использовании модели с параметрами по умолчанию.

### Optuna (продвинутая оптимизация)

In [20]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 310, 10)
  max_depth = trial.suggest_int('max_depth', 15, 40, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 3, 7, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=42)
  # обучаем модель
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train))

  return score

In [21]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

[I 2023-07-18 16:35:06,215] A new study created in memory with name: RandomForestClassifier
[I 2023-07-18 16:35:10,275] Trial 0 finished with value: 0.9728078215704247 and parameters: {'n_estimators': 150, 'max_depth': 24, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.9728078215704247.
[I 2023-07-18 16:35:16,838] Trial 1 finished with value: 0.923076923076923 and parameters: {'n_estimators': 310, 'max_depth': 23, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.9728078215704247.
[I 2023-07-18 16:35:21,142] Trial 2 finished with value: 0.9242930982061417 and parameters: {'n_estimators': 210, 'max_depth': 20, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.9728078215704247.
[I 2023-07-18 16:35:27,152] Trial 3 finished with value: 0.9300911854103344 and parameters: {'n_estimators': 260, 'max_depth': 19, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.9728078215704247.
[I 2023-07-18 16:35:31,434] Trial 4 finished with value: 0.923076923076923 and parameters: {'n_es

CPU times: total: 1min 22s
Wall time: 1min 26s


In [22]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state)
model.fit(X_train, y_train)
#y_train_pred = model.predict(X_train)
#print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.82


При помощи Optuna также не удалось увеличить метрику f1, она имеет такое же значение как и у модели с параметрами по умолчанию.
Для улучшения (максимизации) значения метрики f1 необходимо прододжить подбор параметров модели случайного леса.